In [ ]:
import pandas as pd

fake = pd.read_csv("/content/Fake.csv")


In [ ]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
true = pd.read_csv("/content/True.csv")
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
true['label'] = 1
fake['label'] = 0

df = pd.concat([fake, true] , axis=0)
df = df.sample(frac = 1).reset_index(drop=True)


In [ ]:
df.head()

,title,text,subject,date,label
0,WHY THIS AMERICAN Feels Safer With An ISIS Fla...,"This is hysterical, but sadly it s not far-fet...",left-news,"Mar 4, 2016",0
1,Brazil's Congress blocks corruption charges ag...,BRASILIA (Reuters) - The lower house of Brazil...,worldnews,"October 25, 2017",1
2,Hollywood’s Secret Conservative Club Shuts Do...,"A highly exclusive, highly secretive club orga...",News,"April 22, 2016",0
3,Trump Whines That The Media Wouldn’t Film His...,Trump s desperation to convince people that th...,News,"September 22, 2017",0
4,White House Takes Its War On The Press To A T...,Donald Trump and his team are no fans of the f...,News,"February 24, 2017",0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [ ]:
df['content'] = df['title'] + " " + df['text']
df.head()

,title,text,subject,date,label,content
0,WHY THIS AMERICAN Feels Safer With An ISIS Fla...,"This is hysterical, but sadly it s not far-fet...",left-news,"Mar 4, 2016",0,WHY THIS AMERICAN Feels Safer With An ISIS Fla...
1,Brazil's Congress blocks corruption charges ag...,BRASILIA (Reuters) - The lower house of Brazil...,worldnews,"October 25, 2017",1,Brazil's Congress blocks corruption charges ag...
2,Hollywood’s Secret Conservative Club Shuts Do...,"A highly exclusive, highly secretive club orga...",News,"April 22, 2016",0,Hollywood’s Secret Conservative Club Shuts Do...
3,Trump Whines That The Media Wouldn’t Film His...,Trump s desperation to convince people that th...,News,"September 22, 2017",0,Trump Whines That The Media Wouldn’t Film His...
4,White House Takes Its War On The Press To A T...,Donald Trump and his team are no fans of the f...,News,"February 24, 2017",0,White House Takes Its War On The Press To A T...


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
ps = PorterStemmer()

stop_words = set(stopwords.words('english'))

def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-z]', ' ', text)
  words = text.split()
  words = [ps.stem(word) for word in words if word not in stop_words]
  return ' '.join(words)

df['content_cleaned'] = df['content'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features= 5000,
    ngram_range=(1,2)
)

X = vectorizer.fit_transform(df['content_cleaned'])
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9893095768374165
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4725
           1       0.99      0.99      0.99      4255

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
def predict_news(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    return "REAL 📰" if model.predict(vec)[0] == 1 else "FAKE 🚨"


In [ ]:
print(predict_news("The government announced a new education policy today"))


FAKE 🚨


In [ ]:
print(predict_news(
    "Scientists at a leading university have discovered a new method to improve battery efficiency, according to a peer-reviewed study."
))


FAKE 🚨


In [ ]:
print(predict_news(
    "The central bank raised interest rates by 25 basis points following concerns over inflation."
))


REAL 📰


In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['abadi', 'abandon', 'abba', ..., 'zone', 'zor', 'zuma'],
      dtype=object)

In [ ]:
coefficients = model.coef_[0]
coefficients

array([ 0.25514064, -0.0731015 ,  0.63284896, ...,  0.1783522 ,
        0.51584814,  1.26787021])

In [ ]:
import pandas as pd

coef_df = pd.DataFrame({
    'word': feature_names,
    'coefficient': coefficients
})


In [ ]:
top_fake = coef_df.sort_values(by='coefficient').head(20)
print(top_fake)


              word  coefficient
4748           via    -9.629094
4762         video    -8.570167
4713            us    -6.916177
2044          imag    -5.899997
3278  presid trump    -5.444126
1793           gop    -5.443479
3448          read    -5.392069
1944       hillari    -5.229059
2760            mr    -4.605997
2923         obama    -4.466911
1535   featur imag    -4.283754
159        america    -4.023580
1534        featur    -3.968277
4896          wire    -3.925629
4829         watch    -3.887195
1415          even    -3.798927
786            com    -3.702541
2046      imag via    -3.645297
162       american    -3.559644
3537           rep    -3.530716


In [ ]:
top_real = coef_df.sort_values(by='coefficient', ascending=False).head(20)
print(top_real)


                   word  coefficient
3608             reuter    21.800000
3722               said    14.656791
4825  washington reuter     9.027780
3270      presid donald     5.161074
4822         washington     4.796029
4847          wednesday     4.590027
3612      reuter presid     4.540826
4620            tuesday     4.487716
4430           thursday     3.991831
1700             friday     3.579270
3283         presidenti     3.365136
2686             minist     3.361912
2902                nov     3.308276
2728             monday     3.118506
3267      presid barack     2.997727
3755     said statement     2.898281
4460        told report     2.831329
1285                edt     2.805283
4461        told reuter     2.643408
4110          spokesman     2.625680
